# TO DO

- multiple runs on different processes

In [1]:
#Import libraries
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
#os.environ["THEANO_FLAGS"] = "floatX=float32,device=cpu"
import copy
import numpy as np
np.random.seed(786)
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_string_dtype
import sys

import re
import unicodedata
import string
import math
import gc
import time
from time import time
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from tqdm import tqdm
tqdm.pandas(tqdm)

from fastcache import clru_cache as lru_cache
import threading
import multiprocessing
from multiprocessing import Pool, cpu_count
num_partitions = 8
num_cores = 4

import wordbatch
from  wordbatch.extractors import WordSeq, WordBag, WordHash
from wordbatch.models import FTRL, FM_FTRL, NN_ReLU_H1, NN_ReLU_H2

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, RobustScaler, MaxAbsScaler, QuantileTransformer, OneHotEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin

from scipy.sparse.csr import csr_matrix
from scipy.sparse import hstack
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Reshape, concatenate, Dense, GlobalAveragePooling1D, BatchNormalization, Flatten, Dropout, PReLU, LeakyReLU, Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.text import Tokenizer
from keras.callbacks import Callback, ModelCheckpoint
from keras.optimizers import Adam, RMSprop, Nadam
import keras.backend as K
import tensorflow as tf

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
K.backend()

'tensorflow'

In [3]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_sklearn = make_scorer(rmse, greater_is_better=False)
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

stop_words = ['a', 'an', 'this', 'is', 'the', 'of', 'for']

def unicodeToAscii(series):
    return  series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))

def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))
    def one_xlat(match):
        return adict[match.group(0)]
    return rx.sub(one_xlat, text)

# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)

    replace_dict = {r" one ": " 1 ",
                    r" two ": r" 2 ",
                    r" three ": " 3 ",
                    r" four " : " 4 ", 
                    r" five " : " 5 ",
                    r" six "  : " 6 ", 
                    r" seven ": " 7 ",   
                    r"lily jade": "lilyjade", 
                    r"rae dunn cookies": "raedunncookie",
                    r"rae dunn cookie": "raedunncookie",
                    r"hatchimals" : "hatchimal", 
                    r"virtual reality" : "vr", 
                    r" vs " : " victorias secret ",
                    r" mk " : " michael kors ", 
                    r" victoria secret " : " victorias secret ",
                    r"google home" : "googlehome", 
                    r"16 gb" : "16gb ", 
                    r"256 gb" : "256gb ",
                    r"32 gb" : "32gb ", 
                    r"14k gold" : '14kgold', 
                    r"14 gold" : '14kgold',
                    r"14 k gold" : '14kgold',
                    r"lululemon bags" : 'lululemonbags',
                    r"controller skin" : 'controllerskin',
                    r"watch box" : 'watchbox',
                    r"blaze band" : 'blazeband', 
                    r"vault boy" : 'vaultboy',
                    r"lash boost" : 'lashboost', 
                    r"64 g " : '64gb ',
                    r"32 g " : '32gb',
                    r"go pro hero" : 'goprohero', 
                    r"gopro hero" : 'goprohero', 
                    r"nmd r " : 'nmdr ', 
                    r"nmd r1 " : 'nmdr ', 
                    r"nmds r " : 'nmdr ',  
                    r"private sale" : 'privatesale', 
                    r"vutton" : 'vuitton',
                    r"louis vuitton eva" : 'louisvuittoneva',
                    r"apple watch" : 'applewatch'}
    rx = re.compile('|'.join(map(re.escape, replace_dict)))
    def one_xlat(match):
        return replace_dict[match.group(0)]
    
    series = series.str.lower()
    series = series.str.replace(r"'", "")
    series = series.str.replace(r"-", "")
    series = series.str.replace(r"[^0-9a-zA-Z]+", " ")
    series = series.str.replace(r"(?=\w{1,2})iphone ", "iphone")
    series = series.str.replace(r"(?=\w{1,2})galaxy ", "galaxy")
    series = series.str.replace(rx, one_xlat)

    return series


def _normalize_and_ngrams(sent, ngram):
    input_list = sent.split()
    input_list = [word for word in input_list if word not in stop_words]
    #s = input_list.copy()
    #for i in range(2, ngrams+1):
    #    s += [' '.join(input_list[j:j+i]) for j in range(len(input_list)-i + 1)]
        #s += list((zip(*[input_list[j:] for j in range(i)])))
    s = [''.join(input_list[i:i+ngram]) for i in range(len(input_list))]
    return ' '.join(s[:-1])

stemmer = PorterStemmer()


@lru_cache(1024)
def stem(s):
    return stemmer.stem(s)

whitespace = re.compile(r'\s+')
non_letter = re.compile(r'\W+')

def tokenize(text):
    #text = normalizeString(str(text))
    text = non_letter.sub(' ', text)

    tokens = []

    for t in text.split():
        #t = stem(t)
        tokens.append(t)

    return tokens

class Tokenizer:
    def __init__(self, min_df=10, tokenizer=str.split):
        self.min_df = min_df
        self.tokenizer = tokenizer
        self.doc_freq = None
        self.vocab = None
        self.vocab_idx = None
        self.max_len = None

    def fit_transform(self, texts):
        tokenized = []
        doc_freq = Counter()
        n = len(texts)

        for text in texts:
            sentence = self.tokenizer(text)
            tokenized.append(sentence)
            doc_freq.update(set(sentence))

        vocab = sorted([t for (t, c) in doc_freq.items() if c >= self.min_df])
        vocab_idx = {t: (i + 1) for (i, t) in enumerate(vocab)}
        doc_freq = [doc_freq[t] for t in vocab]

        self.doc_freq = doc_freq
        self.vocab = vocab
        self.vocab_idx = vocab_idx

        max_len = 0
        result_list = []
        for text in tokenized:
            text = self.text_to_idx(text)
            max_len = max(max_len, len(text))
            result_list.append(text)

        self.max_len = max_len
        result = np.zeros(shape=(n, max_len), dtype=np.int32)
        for i in range(n):
            text = result_list[i]
            result[i, :len(text)] = text

        return result    

    def text_to_idx(self, tokenized):
        return [self.vocab_idx[t] for t in tokenized if t in self.vocab_idx]

    def transform(self, texts):
        n = len(texts)
        result = np.zeros(shape=(n, self.max_len), dtype=np.int32)

        for i in range(n):
            text = self.tokenizer(texts[i])
            text = self.text_to_idx(text)[:self.max_len]
            result[i, :len(text)] = text

        return result
    
    def vocabulary_size(self):
        return len(self.vocab) + 1


def get_cat_1(x): return str(x).split('/')[0]
def get_cat_2(x): return str(x).split('/')[1] if len(str(x).split('/')) > 1 else -1
def get_cat_3(x): return ' '.join(str(x).split('/')[2:]) if len(str(x).split('/')) > 2 else -1

def applycat1(df): 
    return df['category_name'].progress_apply(get_cat_1)
    

def applycat2(df): 
    return df['category_name'].progress_apply(get_cat_2)
    

def applycat3(df): 
    return df['category_name'].progress_apply(get_cat_3)

def get_words(series): return series.progress_apply(lambda x: len(str(x).split()))

def get_chars(series): return series.progress_apply(lambda x: len(str(x)))

def get_tokens(series): return np.sum(np.array(series.tolist()) > 0, axis=1)

def isphonecase(series): return series.str.contains(' case ', flags=re.IGNORECASE).astype(int)

def isiphone6(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('6|six', flags=re.IGNORECASE) &
                        ~(series.str.contains('plus|\+', flags=re.IGNORECASE)) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isiphone6p(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('6|six', flags=re.IGNORECASE) &
                        series.str.contains('plus|\+', flags=re.IGNORECASE) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isiphone5(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('5|five', flags=re.IGNORECASE) &
                        ~(series.str.contains('plus|\+', flags=re.IGNORECASE)) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isiphone5p(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('5|five', flags=re.IGNORECASE) &
                        series.str.contains('plus|\+', flags=re.IGNORECASE) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isiphone7(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('7|seven', flags=re.IGNORECASE) &
                        ~(series.str.contains('plus|\+', flags=re.IGNORECASE)) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isiphone7p(series): return (series.str.contains('iphone', flags=re.IGNORECASE) & 
                        series.str.contains('7|seven', flags=re.IGNORECASE) &
                        series.str.contains('plus|\+', flags=re.IGNORECASE) &
                                ~(series.str.contains('case', flags=re.IGNORECASE)) ).astype(int)

def isunlocked(series): return series.str.contains('unlocked', flags=re.IGNORECASE).astype(int)

def plussigns(series): return series.apply(lambda x: sum([(s == '+') | (s == '➕') for s in str(x)]))

def andsigns(series): return series.apply(lambda x: sum([(s == '&') | (s == ' and ') for s in str(x)]))

def commas(series): return series.apply(lambda x: sum([s == ',' for s in str(x)]))

def add_ngrams(text, ngram=2):
    word_list = normalizeString(text).split(' ')
    out_list = [''.join(word_list[i:i+ngram]) for i in range(len(word_list))]
    return ' '.join(out_list[:-1])

def get_2grams(series): return series.apply(lambda x: _normalize_and_ngrams(str(x), 2))

def norm3grams(s): return _normalize_and_ngrams(s, 3)

def applyname(series): return series.progress_apply(norm3grams)

def index2sent1(x, name_vocab): return indexesFromSentence(name_vocab, x, 3, 10)

def name2index(series): return series.progress_apply(lambda x: index2sent1(x, name_vocab))

def norm2grams(s): return _normalize_and_ngrams(s, 1)

def applydesc(series):return series.progress_apply(norm2grams)

def index2sent2(x, desc_vocab): return indexesFromSentence(desc_vocab, x, 1, 80)

def desc2index(series): return series.progress_apply(lambda x: index2sent2(x, desc_vocab))

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def cutting(dataset):
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category1 = dataset['category_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
    dataset.loc[~dataset['category_name'].isin(pop_category1), 'category_name'] = 'missing'

    
def generate_features(data):
        data["item_description"] = data["item_description"].replace("No description yet", "missing")
        
        data['cat1'] = parallelize_dataframe(data[['category_name']], applycat1)
        data['cat2'] = parallelize_dataframe(data[['category_name']], applycat2)
        data['cat3'] = parallelize_dataframe(data[['category_name']], applycat3)
        data.fillna(-1, inplace=True)
        
        print("Getting word/char len features")
        data['desc_words'] = parallelize_dataframe(data['item_description'], get_words)
        data['desc_chars'] = parallelize_dataframe(data['item_description'], get_chars)
        data['name_words'] = parallelize_dataframe(data['name'], get_words)
        data['name_chars'] = parallelize_dataframe(data['name'], get_chars)
        
        
        print("Get iphone features")
        data['iphone_case'] = parallelize_dataframe(data['name'], isphonecase)
        data['iphone6'] = parallelize_dataframe(data['name'], isiphone6)
        data['iphone6p'] = parallelize_dataframe(data['name'], isiphone6p)
        data['iphone5'] = parallelize_dataframe(data['name'], isiphone5)
        data['iphone5p'] = parallelize_dataframe(data['name'], isiphone5p)
        data['iphone7'] = parallelize_dataframe(data['name'], isiphone7)
        data['iphone7p'] = parallelize_dataframe(data['name'], isiphone7p)
        data['unlocked_phone'] = parallelize_dataframe(data['name'], isunlocked)
        
        all_brands = set(data['brand_name'].values)
        def brandfinder(line):
            brand = line[0]
            name = line[1]
            namesplit = name.split(' ')
            if brand == 'missing':
                for x in namesplit:
                    if x in all_brands:
                        return x
            #else:
            #    return brand
            #if name in all_brands:
            #    return name
            return brand
        #data['brand_name2'] = data[['brand_name','name']].apply(brandfinder, axis = 1)   
        
        #print("Label encoding features")
        #cat_cols = ['category_name', 'brand_name', 'brand_name2', 'cat1', 'cat2', 'cat3', 'item_condition_id']
        #for col in cat_cols:
        #    data[col] = LabelEncoder().fit_transform(data[col].astype(str)) + 1
            
        print("Get count features")
        data['brand_counts'] = data.brand_name.map(data["brand_name"].value_counts()).fillna(0).astype(int)

        data['cat_counts'] = data.category_name.map(data["category_name"].value_counts()).fillna(0).astype(int)
        
        data['cat1_counts'] = data.cat1.map(data["cat1"].value_counts()).fillna(0).astype(int)

        data['cat2_counts'] = data.cat2.map(data["cat2"].value_counts()).fillna(0).astype(int)

        data['cat3_counts'] = data.cat3.map(data["cat3"].value_counts()).fillna(0).astype(int)
  
        
        print("Getting punct related features")
        data["plus_counts"] = parallelize_dataframe(data["item_description"], plussigns)
        data["ands_counts"] = parallelize_dataframe(data["item_description"], andsigns)
        data["comma_counts"] = parallelize_dataframe(data["item_description"], commas)
        data["all_counts"] = data["plus_counts"] + data["ands_counts"] + data["comma_counts"]
        
        #for col in ["name", "item_description"]:
        #    data[col] = data[col].str.replace("'", '').replace('-', '').progress_apply(unicodeToAscii)
        #    data[col] = data[col].progress_apply(remove_puncts)
        
        data["brand_cat"] = data["brand_name"].astype(str) + ' ' + data["category_name"].astype(str)
        data["category_shipping"] = data["category_name"].astype(str) + ' ' + data["shipping"].astype(str)
        
        data['brand_cat_counts'] = data.brand_cat.map(data["brand_cat"].value_counts()).fillna(0).astype(int)
        
        num_cols =  ["desc_words", "desc_chars", "name_words", "name_chars", "plus_counts", 
                    "ands_counts", "comma_counts", "all_counts", "brand_counts", "cat_counts", "cat1_counts", 
                   "cat2_counts", "cat3_counts", "brand_cat_counts"]
        data[num_cols]  = QuantileTransformer(output_distribution='normal', random_state=1).fit_transform(data[num_cols])


In [4]:
%%time
in_path = "../input/"
train_data = pd.read_table(os.path.join(in_path, 'train.tsv'))
test_data  = pd.read_table(os.path.join(in_path, 'test.tsv'))
train_data = train_data.loc[(train_data.price > 2)].reset_index(drop=True)

train_rows = len(train_data)
data = pd.concat([train_data, test_data], ignore_index=True)

print(train_data.shape, test_data.shape)
print(train_data.memory_usage().sum().sum()/10**6, test_data.memory_usage().sum().sum()/10**6)
print(data.memory_usage().sum().sum()/10**6)

(1481661, 8) (693359, 7)
94.826384 38.828184
156.60152
CPU times: user 6.6 s, sys: 517 ms, total: 7.12 s
Wall time: 7.16 s


In [5]:
gc.collect()

0

In [6]:
%%time
NUM_BRANDS = 4500
NUM_CATEGORIES = 1200

cutting(data)
generate_features(data)

print(data.shape)
data.head()

100%|██████████| 271877/271877 [00:00<00:00, 456293.07it/s]


Getting word/char len features


100%|██████████| 271877/271877 [00:00<00:00, 764326.61it/s]


Get iphone features
Get count features
Getting punct related features
(2175020, 36)
CPU times: user 29.8 s, sys: 9.89 s, total: 39.7 s
Wall time: 1min 16s


In [7]:
gc.collect()
print(train_data.memory_usage().sum().sum()/10**6, test_data.memory_usage().sum().sum()/10**6)
print(data.memory_usage().sum().sum()/10**6)

94.826384 38.828184
626.40584


In [8]:
%%time
for col in ['name', 'item_description']:
    train_data[col] = normalizeString(train_data[col])
    test_data[col] = normalizeString(test_data[col])


CPU times: user 1min 33s, sys: 234 ms, total: 1min 34s
Wall time: 1min 34s


In [9]:
train_data['item_description'].sample(2).values

array(['brand new without tag price is firm material polyester cotton color green mint as pictured size 190 x 85 cm 74 x 34 inches this is adult size fits most average adult features super cozy comfy snuggle woolen yarn blanket for adult and girls with beautiful knitted pattern to complement your living room or bedroom decor blanket features with fish scales pattern open mermaid fins with tassels handcrafted and exquisite ideal gifts for girls and women great to wrap on the sofa couch bed or car for reading watching tv or nap sleep',
       'size18 in plus size'], dtype=object)

In [10]:
%%time
print('processing categories...')
X_cats = {}
X_test_cats = {}
for col in ["brand_name", "category_name", "cat1", "cat2", "cat3", "brand_cat", "category_shipping", "item_condition_id"]:
    data[col] = data[col].astype('category').cat.codes + 1
    X_cats[col] = data.loc[:train_rows-1, col].astype(np.int32).values.reshape(-1,1)
    X_test_cats[col] = data.loc[train_rows:, col].astype(np.int32).values.reshape(-1,1)
    
X_conts = []
X_test_conts = []
for col in ['shipping', 'desc_words', 'desc_chars',
            'name_words', 'name_chars',  'iphone_case', 'iphone6', 'iphone6p', 
            'iphone5', 'iphone5p', 'iphone7', 'iphone7p', 'unlocked_phone', 
            'brand_counts', 'cat_counts', 'cat1_counts', 
            'cat2_counts', 'cat3_counts', 'plus_counts', 'ands_counts', 
            'comma_counts', 'all_counts']:
    print(col)
    X_conts.append(data.loc[:train_rows-1, col].replace('missing', 0).astype(np.float32).values)
    X_test_conts.append(data.loc[train_rows:, col].replace('missing', 0).astype(np.float32).values)

X_conts = np.array(X_conts).T
X_test_conts = np.array(X_test_conts).T

print(X_conts.shape, X_test_conts.shape)

processing categories...
shipping
desc_words
desc_chars
name_words
name_chars
iphone_case
iphone6
iphone6p
iphone5
iphone5p
iphone7
iphone7p
unlocked_phone
brand_counts
cat_counts
cat1_counts
cat2_counts
cat3_counts
plus_counts
ands_counts
comma_counts
all_counts
(1481661, 22) (693359, 22)
CPU times: user 9.05 s, sys: 741 ms, total: 9.79 s
Wall time: 9.76 s


In [11]:
del data
gc.collect()

497

In [12]:
#%%time
#test_data['name'] = test_data['name'].str.replace("nmd(s){0,1}(\s){0,1}(r){0,1}(1){0,1}(\s|$)", 'nmdr ')

In [13]:
%%time
print('processing title...')
name_num_col = 7
#name_voc_size = 26000
name_tok = Tokenizer(min_df=5)
#name_tok.fit_on_texts(train_data.name)
#X_name = pad_sequences(name_tok.texts_to_sequences(train_data.name), maxlen=name_num_col, truncating='post')
#X_test_name = pad_sequences(name_tok.texts_to_sequences(test_data.name), maxlen=name_num_col, truncating='post')
X_name = name_tok.fit_transform(train_data.name)
X_test_name = name_tok.transform(test_data.name)

X_name = X_name[:, :name_num_col]
X_test_name = X_test_name[:, :name_num_col]

name_voc_size = name_tok.vocabulary_size()
print(name_voc_size)

processing title...
26399
CPU times: user 22.2 s, sys: 176 ms, total: 22.4 s
Wall time: 22.4 s


In [14]:
%%time
print('processing description...')

desc_num_col = 70
#desc_voc_size = 50000

#desc_tok = Tokenizer(num_words=desc_voc_size)
#desc_tok.fit_on_texts(train_data.item_description)

#X_desc = pad_sequences(desc_tok.texts_to_sequences(train_data.item_description.astype(str)), maxlen=desc_num_col, truncating='post')
#X_test_desc = pad_sequences(desc_tok.texts_to_sequences(test_data.item_description.astype(str)), maxlen=desc_num_col, truncating='post')

desc_tok = Tokenizer(min_df=10)
X_desc = desc_tok.fit_transform(train_data["item_description"])
X_test_desc = desc_tok.transform(test_data["item_description"])
X_desc = X_desc[:, :desc_num_col]
X_test_desc = X_test_desc[:, :desc_num_col]
desc_voc_size = desc_tok.vocabulary_size()
print(desc_voc_size)

processing description...
34245
CPU times: user 47.8 s, sys: 1.27 s, total: 49.1 s
Wall time: 49.1 s


In [15]:
%%time
print('processing description2...')

train_data['item_desc2gram'] = parallelize_dataframe(train_data["item_description"], get_2grams)
test_data['item_desc2gram'] = get_2grams(test_data["item_description"])

desc2_num_col = 20
#desc2_voc_size = 20000
#desc_tok2 = Tokenizer(num_words= desc2_voc_size)
#desc_tok2.fit_on_texts(train_data.item_desc2gram)

#X_desc2 = pad_sequences(desc_tok2.texts_to_sequences(train_data.item_desc2gram), maxlen=desc2_num_col, truncating='post')
#X_test_desc2 = pad_sequences(desc_tok2.texts_to_sequences(test_data.item_desc2gram), maxlen=desc2_num_col, truncating='post')

desc_tok2 = Tokenizer(min_df=200)
X_desc2 = desc_tok2.fit_transform(train_data['item_desc2gram'])
X_test_desc2 = desc_tok2.transform(test_data['item_desc2gram'])
X_desc2 = X_desc2[:, :desc2_num_col]
X_test_desc2 = X_test_desc2[:, :desc2_num_col]

desc2_voc_size = desc_tok2.vocabulary_size()
print(desc2_voc_size)

processing description2...
18200
CPU times: user 58.2 s, sys: 2.33 s, total: 1min
Wall time: 1min 4s


In [16]:
%%time
print('processing item name...')
itemname_num_col = 15
#X_itemname = pad_sequences(name_tok.texts_to_sequences(train_data.item_description.astype(str)), maxlen=itemname_num_col, truncating='post')
#X_test_itemname = pad_sequences(name_tok.texts_to_sequences(test_data.item_description.astype(str)), maxlen=itemname_num_col, truncating='post')

X_itemname = name_tok.transform(train_data['item_description'])
X_test_itemname = name_tok.transform(test_data['item_description'])
X_itemname = X_itemname[:, :itemname_num_col]
X_test_itemname = X_test_itemname[:, :itemname_num_col]
name_voc_size = name_tok.vocabulary_size()
print(name_voc_size)

processing item name...
26399
CPU times: user 53.8 s, sys: 51.7 ms, total: 53.8 s
Wall time: 53.8 s


In [17]:
#Lets get sizes
gc.collect()
print(X_name.nbytes/10**6, X_test_name.nbytes/10**6)
print(X_desc.nbytes/10**6, X_test_desc.nbytes/10**6)
print(X_desc2.nbytes/10**6, X_test_desc2.nbytes/10**6)
print(X_itemname.nbytes/10**6, X_test_itemname.nbytes/10**6)
print(X_conts.nbytes/10**6, X_test_conts.nbytes/10**6)
print(train_data.memory_usage().sum().sum()/10**6, test_data.memory_usage().sum().sum()/10**6)

41.486508 19.414052
414.86508 194.14052
118.53288 55.46872
88.89966 41.60154
130.386168 61.015592
106.679672 44.375056


In [18]:
#del name_tok, desc_tok, desc_tok2

In [22]:
#def keras model
y = np.log1p(train_data.price).values.reshape(-1,1)
cvlist = list(KFold(40, random_state=1).split(X_desc))
std = np.std(y)
mean = np.mean(y)
ynorm = (y - mean)/std

name_seq_len = X_name.shape[1]
name_embeddings_dim = 50

itemname_embeddings_dim = 30

desc_seq_len = X_desc.shape[1]
desc_embeddings_dim = 50

desc2_seq_len = X_desc2.shape[1]
desc2_embeddings_dim = 30

brand_voc_size = 5600
brand_embeddings_dim = 50

cat_voc_size = 1500
cat_embeddings_dim = 40

cat1_voc_size = 16
cat1_embeddings_dim = 6

cat2_voc_size = 121
cat2_embeddings_dim = 15

cat3_voc_size = 900
cat3_embeddings_dim = 20

brandcat_voc_size = 51000
brandcat_embeddings_dim = 30

catship_voc_size = 3500
catship_embeddings_dim = 20

cond_voc_size = 6
cond_embedding_size = 4

def keras_mercari_model(seed, params):
    #Get all input params
    name_embeddings_dim, desc_embeddings_dim, desc2_embeddings_dim, \
    brand_embeddings_dim, cat_embeddings_dim, cat1_embeddings_dim, \
    cat2_embeddings_dim, cat3_embeddings_dim, cond_embedding_size, \
    brandcat_embeddings_dim, catship_embeddings_dim, itemname_embeddings_dim, \
    name_drop, desc_drop, desc2_drop, brand_drop, cat_drop, cat1_drop, cat2_drop, \
    cat3_drop, cond_drop, brandcat_drop, catship_drop, itemname_drop, \
    dense_dim, lr, lr_decay = params
    
    
    name = Input(shape=(name_seq_len,))
    desc = Input(shape=(desc_seq_len,))
    desc2 = Input(shape=(desc2_seq_len,))
    itemname = Input(shape=(itemname_num_col,))
    brand = Input(shape=(1,))
    cat = Input(shape=(1,))
    cat1 = Input(shape=(1,))
    cat2 = Input(shape=(1,))
    cat3 = Input(shape=(1,))
    brandcat = Input(shape=(1,))
    catship = Input(shape=(1,))
    cond = Input(shape=(1,))
    conts = Input(shape=(X_conts.shape[1],))
    inputs = [name, desc, desc2, itemname, brand, cat, cat1, cat2, cat3, brandcat, catship, cond, conts]
    
    
    embed_name = Embedding(name_voc_size, name_embeddings_dim, embeddings_initializer='he_uniform')
    name = embed_name(name)
    desc = Embedding(desc_voc_size, desc_embeddings_dim, embeddings_initializer='he_uniform')(desc)
    desc2 = Embedding(desc2_voc_size, desc2_embeddings_dim, embeddings_initializer='he_uniform')(desc2)
    itemname = Embedding(name_voc_size, itemname_embeddings_dim, embeddings_initializer='he_uniform')(itemname)
    
    brand = Embedding(brand_voc_size, brand_embeddings_dim, embeddings_initializer='he_uniform')(brand)
    cat = Embedding(cat_voc_size, cat_embeddings_dim, embeddings_initializer='he_uniform')(cat)
    cat1 = Embedding(cat1_voc_size, cat1_embeddings_dim, embeddings_initializer='he_uniform')(cat1)
    cat2 = Embedding(cat2_voc_size, cat2_embeddings_dim, embeddings_initializer='he_uniform')(cat2)
    cat3 = Embedding(cat3_voc_size, cat3_embeddings_dim, embeddings_initializer='he_uniform')(cat3)
    brandcat = Embedding(brandcat_voc_size, brandcat_embeddings_dim, embeddings_initializer='he_uniform')(brandcat)
    catship = Embedding(catship_voc_size, catship_embeddings_dim, embeddings_initializer='he_uniform')(catship)
    cond = Embedding(cond_voc_size, cond_embedding_size, embeddings_initializer='he_uniform')(cond)
    
    #name2 = Conv1D(32, 2, activation='relu', kernel_initializer='he_normal', use_bias=False)(name)
    #name2 = Dropout(0.2)(name2)
    #name2 = MaxPooling1D()(name2)
    #name2 = GlobalAveragePooling1D()(name2)
    
    #name3 = Conv1D(32, 3, activation='relu', kernel_initializer='he_normal', use_bias=False)(name)
    #name3 = Dropout(0.2)(name3)
    #name3 = MaxPooling1D()(name3)
    #name3 = GlobalAveragePooling1D()(name3)
    
    #name2 = concatenate([name2, name3])
    #name2 = GlobalAveragePooling1D()(name2)
    #name3 = Conv1D(name_embeddings_dim, 3, activation='relu', kernel_initializer='he_normal', use_bias=False)(name)
    print(name.shape)
    #name = concatenate([name, name2], 1)
    name = Dropout(name_drop, seed=seed)(name)
    name = GlobalAveragePooling1D()(name)
    
    #name = Flatten()(name)
    
    #desc = Conv1D(20, 3, activation='relu', kernel_initializer='he_normal', use_bias=False)(desc)

    #desc = Flatten()(desc)
    #desc21 = Conv1D(32, 2, activation='relu', kernel_initializer='he_normal', use_bias=False)(desc)
    #desc21 = Dropout(0.4)(desc21)
    #desc21 = MaxPooling1D()(desc21)
    #desc21 = GlobalAveragePooling1D()(desc21)
    
    #desc3 = Conv1D(16, 3, activation='relu', kernel_initializer='he_normal', use_bias=False)(desc)
    #desc3 = Dropout(0.6)(desc3)
    #desc3 = MaxPooling1D()(desc3)
    #desc3 = GlobalAveragePooling1D()(desc3)
    
    #desc23 = concatenate([desc21,desc3])
    
    desc = Dropout(desc_drop, seed=seed)(desc)
    desc = GlobalAveragePooling1D()(desc)
    #
    desc2 = Dropout(desc2_drop, seed=seed)(desc2)
    desc2 = GlobalAveragePooling1D()(desc2)
    #desc2 = Flatten()(desc2)
    
    itemname = Dropout(itemname_drop, seed=seed)(itemname)
    itemname = GlobalAveragePooling1D()(itemname)
    #itemname = Flatten()(itemname)
    
    brandcat = Dropout(brandcat_drop, seed=seed)(brandcat)
    catship = Dropout(catship_drop, seed=seed)(catship)
    
    #conts = Dropout(0.02, seed=786)(conts)
    cat = Dropout(cat_drop, seed=seed)(cat)
    cat1 = Dropout(cat1_drop, seed=seed)(cat)
    cat2 = Dropout(cat2_drop, seed=seed)(cat)
    cat3 = Dropout(cat3_drop, seed=seed)(cat)
    brand = Dropout(brand_drop, seed=seed)(cat)
    
    brand = Flatten()(brand)
    cat = Flatten() (cat)
    cat1 = Flatten()(cat1)
    cat2 = Flatten()(cat2)
    cat3 = Flatten()(cat3)
    brandcat = Flatten()(brandcat)
    catship = Flatten()(catship)
    cond = Flatten()(cond)
    
    out = concatenate([name, 
                       #name2,
                       desc, 
                       desc2,
                       #desc3,
                       itemname, 
                       brand, 
                       cat, 
                       cat1, 
                       cat2,
                       cat3, 
                       brandcat, 
                       catship, 
                       cond, 
                       #conts
                      ])
    print(out.shape)
    out = BatchNormalization(momentum=0.9)(out)
    out = Dropout(0.00, seed=786)(out)
    
    out = Dense(dense_dim, activation='selu',kernel_initializer='he_normal')(out)
    out = PReLU()(out)
    out = BatchNormalization(momentum=0.99)(out)
    out = Dropout(0.00, seed=786)(out)
    out = Dense(1, kernel_initializer='normal')(out)
    
    model = Model(inputs, out)
    opt = RMSprop(lr=lr, decay=lr_decay, clipvalue=0.1)
    model.compile(optimizer=opt, loss="mse" )
    return model

In [23]:
num_cores = 1
GPU= True
CPU= False
if GPU:
    num_GPU = 1
    num_CPU = 1
if CPU:
    num_CPU = 1
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)



In [24]:
class TestCallback(Callback):
    def __init__(self, x_val, x_test):
        self.x_val = x_val
        self.x_test = x_test
        self.val_preds = []
        self.test_preds = []
        
    def on_epoch_end(self, epoch, logs={}):
        y_val_preds = self.model.predict(self.x_val)
        y_test_preds = self.model.predict(self.x_test)
        self.val_preds.append(y_val_preds)
        self.test_preds.append(y_test_preds)




In [28]:
params = [64, 64, 32, 32, 
          32, 8, 16, 24, 5,
          32, 32, 24, 
         0.05, 0.3, 0.05, 0.05,
          0.1, 0.1, 0.1, 0.1,
          0.0, 0.3, 0.1, 0.2,
         200, 0.009, 0.006]
X = [X_name, X_desc, X_desc2, X_itemname, X_cats["brand_name"], X_cats["category_name"],
    X_cats["cat1"], X_cats["cat2"], X_cats["cat3"], X_cats["brand_cat"],
    X_cats["category_shipping"], X_cats["item_condition_id"], X_conts]

X_test = [X_test_name, X_test_desc, X_test_desc2, X_test_itemname, X_test_cats["brand_name"], X_test_cats["category_name"],
    X_test_cats["cat1"], X_test_cats["cat2"], X_test_cats["cat3"], X_test_cats["brand_cat"],
    X_test_cats["category_shipping"], X_test_cats["item_condition_id"], X_test_conts]

def train_model(params):
    y_trues = []
    y_preds = []
    for seed in [1,2,3,4,5,6,7,8,9,10]:
        batchsize=2000
        epochs=4
        np.random.seed(seed)
        tf.set_random_seed(seed)

        model = keras_mercari_model(seed, params)

        train_idx, val_idx= cvlist[seed]


        X_tr = [x[train_idx] for x in X]
        X_val = [x[val_idx] for x in X]

        #val_store = TestCallback(X_val, X_test)
        gc.collect()
        model.fit(X_tr, y[train_idx], batch_size=batchsize, epochs=epochs,
                                       verbose=1,
                                      validation_data=(X_val, y[val_idx]), shuffle=True,
                                      #callbacks= [val_store]
                 )
        y_val = y[val_idx,0]
        y_pred = model.predict(X_val)[:, 0]
        print(np.sqrt(metrics.mean_squared_error(y_val, y_pred)))
        
        y_trues.extend(y_val)
        y_preds.extend(y_pred)
        K.clear_session()
    score = np.sqrt(metrics.mean_squared_error(y_trues, y_preds))
    print("Score with params {} is {}".format(params, score))
    return score


In [29]:
import skopt
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Real, Integer, Categorical
gc.collect()

5250

In [ ]:
space = [Integer(16, 128), #name='name_embed'),
         Integer(16, 128),# name='desc_embed'),
         Integer(16, 96),# name='desc2_embed'),
         Integer(16, 128),# name='brand_embed'),
         Integer(16, 128),# name='cat_embed'),
         Integer(4, 32),# name='cat1_embed'),
         Integer(4, 64),# name='cat2_embed'),
         Integer(4, 64),# name='cat3_embed'),
         Integer(2, 8),# name='cond_embed'),
         Integer(8, 64),# name='brandcat_embed'),
         Integer(8, 64),# name='catship_embed'),
         Integer(8, 64),# name='itemname_embed'),
         Real(0.00, 0.2, "uniform"), #name_drop'),
         Real(0.00, 0.5, "uniform"), #desc_drop'),
         Real(0.00, 0.5, "uniform"), #desc2_drop'),
         Real(0.00, 0.2, "uniform"), #brand_drop),
         Real(0.00, 0.2, "uniform"), #cat_drop),
         Real(0.00, 0.2, "uniform"), #cat1_drop),
         Real(0.00, 0.2, "uniform"), #cat2_drop),
         Real(0.00, 0.2, "uniform"), #cat3_drop),
         Real(0.00, 0.1, "uniform"), #cond_drop),
         Real(0.00, 0.5, "uniform"), #brandcat_drop),
         Real(0.00, 0.4, "uniform"), #catship_drop,
         Real(0.00, 0.5, "uniform"), #itemname),
         Integer(64, 512),# 'dense_dim'),
         Real(0.002, 0.015, "uniform"), #lr_decay),
         Real(0.001, 0.015, "uniform"), #lr_decay),
        ]

res_gp = gbrt_minimize(train_model, space, n_calls=100, random_state=0)

(?, 7, 60)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 22s 16us/step - loss: 0.9105 - val_loss: 0.1981
Epoch 2/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1806 - val_loss: 0.1837
Epoch 3/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1649 - val_loss: 0.1807
Epoch 4/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1563 - val_loss: 0.1756
0.4191008970561012
(?, 7, 60)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.9064 - val_loss: 0.2268
Epoch 2/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1814 - val_loss: 0.1864
Epoch 3/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1656 - val_loss: 0.1807
Epoch 4/4
1444619/1444619 [==============================] - 19s 13us/step - l

1444619/1444619 [==============================] - 16s 11us/step - loss: 0.1475 - val_loss: 0.1718
0.4144977441459289
(?, 7, 65)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 16s 11us/step - loss: 0.8170 - val_loss: 0.2130
Epoch 2/4
1444619/1444619 [==============================] - 16s 11us/step - loss: 0.1785 - val_loss: 0.1859
Epoch 3/4
1444619/1444619 [==============================] - 16s 11us/step - loss: 0.1581 - val_loss: 0.1815
Epoch 4/4
1444619/1444619 [==============================] - 16s 11us/step - loss: 0.1473 - val_loss: 0.1720
0.4147146527269324
(?, 7, 65)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 17s 11us/step - loss: 0.8188 - val_loss: 0.1965
Epoch 2/4
1444619/1444619 [==============================] - 16s 11us/step - loss: 0.1797 - val_loss: 0.1817
Epoch 3/4
1444619/1444619 [==============================] - 16s 11us

1444619/1444619 [==============================] - 17s 12us/step - loss: 0.1632 - val_loss: 0.1810
Epoch 4/4
1444619/1444619 [==============================] - 17s 12us/step - loss: 0.1564 - val_loss: 0.1805
0.4248748024401624
Score with params [73, 51, 27, 62, 98, 31, 4, 18, 5, 61, 20, 50, 0.08772030269246409, 0.4941869190296132, 0.05102240537401405, 0.041775351218966944, 0.032261903576999255, 0.1306216650930797, 0.050658320507956434, 0.09326215457126127, 0.02444255920016028, 0.07948479182275987, 0.044150056465722065, 0.32816479473263677, 369, 0.012118491266336818, 0.01429399474831039] is 0.42203818347200256
(?, 7, 111)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.4037 - val_loss: 0.1896
Epoch 2/4
1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1694 - val_loss: 0.1752
Epoch 3/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1547 - va

1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1621 - val_loss: 0.1804
Epoch 4/4
1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1554 - val_loss: 0.1793
0.4234865021852877
(?, 7, 57)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.3917 - val_loss: 0.1904
Epoch 2/4
1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1744 - val_loss: 0.1825
Epoch 3/4
1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1622 - val_loss: 0.1786
Epoch 4/4
1444619/1444619 [==============================] - 18s 13us/step - loss: 0.1553 - val_loss: 0.1776
0.42136743082061573
(?, 7, 57)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.3891 - val_loss: 0.1927
Epoch 2/4
1444619/1444619 [==============================] - 18s 13u

1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1723 - val_loss: 0.1803
Epoch 3/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1547 - val_loss: 0.1784
Epoch 4/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1448 - val_loss: 0.1751
0.41844508545895726
(?, 7, 68)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 21s 14us/step - loss: 0.5382 - val_loss: 0.1985
Epoch 2/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1723 - val_loss: 0.1800
Epoch 3/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1550 - val_loss: 0.1778
Epoch 4/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1453 - val_loss: 0.1800
0.42429643946011203
Score with params [68, 83, 77, 30, 112, 8, 7, 49, 5, 30, 21, 53, 0.13508781624004723, 0.12489813782748907, 0.1566091659729755, 0.19308324

1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1758 - val_loss: 0.1837
Epoch 3/4
1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1607 - val_loss: 0.1793
Epoch 4/4
1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1521 - val_loss: 0.1764
0.4200361295292199
(?, 7, 18)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 15s 11us/step - loss: 0.3885 - val_loss: 0.1978
Epoch 2/4
1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1751 - val_loss: 0.1808
Epoch 3/4
1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1602 - val_loss: 0.1794
Epoch 4/4
1444619/1444619 [==============================] - 15s 10us/step - loss: 0.1518 - val_loss: 0.1764
0.41995431627884244
(?, 7, 18)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 15s 11u

1444619/1444619 [==============================] - 20s 14us/step - loss: 0.3588 - val_loss: 0.1904
Epoch 2/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1758 - val_loss: 0.1825
Epoch 3/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1634 - val_loss: 0.1799
Epoch 4/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1566 - val_loss: 0.1764
0.4200400239841267
(?, 7, 16)
(?, ?)
Train on 1444619 samples, validate on 37042 samples
Epoch 1/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.3564 - val_loss: 0.2020
Epoch 2/4
1444619/1444619 [==============================] - 20s 14us/step - loss: 0.1743 - val_loss: 0.1834
Epoch 3/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1614 - val_loss: 0.1807
Epoch 4/4
1444619/1444619 [==============================] - 19s 13us/step - loss: 0.1543 - val_loss: 0.1790
0.42308461822082766
(?, 7, 16)
(?, ?)
Train on 14

In [20]:
def train_model(seed):
    config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
            inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
            device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
    session = tf.Session(config=config)
    K.set_session(session)

    #seed=1
    batchsize=1000
    epochs=3
    np.random.seed(seed)
    tf.set_random_seed(seed)


    model = keras_mercari_model(seed)

    train_idx, val_idx= cvlist[0]
    X = [X_name, X_desc, X_desc2, X_itemname, X_cats["brand_name"], X_cats["category_name"],
        X_cats["cat1"], X_cats["cat2"], X_cats["cat3"], X_cats["brand_cat"],
        X_cats["category_shipping"], X_cats["item_condition_id"], X_conts]

    X_test = [X_test_name, X_test_desc, X_test_desc2, X_test_itemname, X_test_cats["brand_name"], X_test_cats["category_name"],
        X_test_cats["cat1"], X_test_cats["cat2"], X_test_cats["cat3"], X_test_cats["brand_cat"],
        X_test_cats["category_shipping"], X_test_cats["item_condition_id"], X_test_conts]

    X_tr = [x[train_idx] for x in X]
    X_val = [x[val_idx] for x in X]

    val_store = TestCallback(X_val, X_test)

    model.fit(X_tr, y[train_idx], batch_size=batchsize, epochs=epochs,
                                   verbose=1,
                                  validation_data=(X_val, y[val_idx]), shuffle=True,
                                  #callbacks= [val_store]
             )
    y_val1 = y[val_idx,0]
    y_preds1 = model.predict(X_val)[:, 0]
    print(np.sqrt(metrics.mean_squared_error(y_val1, y_preds1)))

In [22]:
gc.collect()

2179

In [ ]:
if __name__ == "__main__":
    pool = Pool(processes=4)
    lr1s = [0.005, 0.0045, 0.0055, 0.005]
    lr2s = [0.001, 0.001, 0.001, 0.0009]
    results = [pool.apply_async(train_model, args=(np.random.randint(10000000),)) for i in range(3)]
    output = [p.get() for p in results]
    print(output[0][:5], output[1][:5])

(?, 7, 50)
(?, 7, 50)
(?, 7, 50)
(?, ?)
(?, ?)
(?, ?)
Train on 1408408 samples, validate on 74127 samples
Train on 1408408 samples, validate on 74127 samples
Train on 1408408 samples, validate on 74127 samples
Epoch 1/3
Epoch 1/3
Epoch 1/3


In [66]:
train_data.loc[train_data.name.str.contains('([A-Z]){2,}'), ['name', 'brand_name']].sample(10)

/home/mohsin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,name,brand_name
988663,Funko Pop White Lantern Batman NYCC,NaN
1247109,VS 34C Bras,Victoria's Secret
998327,Elta MD UV CLear Facial Sunscreen,NaN
66939,VS Pink Bundle XSS 14 medium,PINK
1095979,LuLaRoe TC Kaleidoscope Rainbow,NaN
420643,Jessica Simpson Purse Clutch 4 AMY,Jessica Simpson
995161,EYESHADOW,NaN
730406,LF Steve Madden limited edition combat,Steve Madden
1123409,DORBZ Teen Titans Robin chase,Funko
735976,Lilly Pulitzer XL Dress,Lilly Pulitzer


In [107]:
# UD - Urban decay
#Eyeshadow
#NYCC
# MD - medium
# FR SHP - same seller - same prices
# hundreds of happy buyers
# item description has price
train_data.loc[train_data.item_description.str.contains('$ \d'),  ['item_description','price']].shape

(1401, 2)

In [114]:
train_data.loc[train_data.name.str.contains("limited edition"), 'price'].mean()

46.4858934169279

In [36]:
y_preds = val_store.val_preds
y_preds = np.concatenate(y_preds, axis=1)
y_preds_mean = y_preds[:, 0]*0.15 + y_preds[:, 1]*0.25 + y_preds[:, 2]*0.6
rmse(y_val1, y_preds_mean)

0.4134122927730147

In [ ]:
import skopt
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Real, Integer, Categorical

space = [Integer(16, 512), #name='name_embed'),
         Integer(16, 512),# name='desc_embed'),
         Integer(16, 256),# name='brand_embed'),
         Integer(16, 256),# name='cat_embed'),
         Integer(64, 512),# name='dense1_dim'),
         Integer(32, 256),# name='dense2_dim'),
         Real(0.001, 0.01, "uniform"), #name='lr1'),
         Real(1e-5, 0.005, "uniform"), #name='lr2'),
         Real(1e-5, 1e-3, "log-uniform"), #name='lr3'),
         Real(0.001, 0.4, "uniform"), #name='name_drop'),
         Real(0.001, 0.5, "uniform"), #name='desc_drop'),
         Real(0.0, 0.05, "uniform"), #name='dense_drop'),
         Integer(10, 30), #name='name_filters3'),
         Integer(10, 30), #name='name_filters2'),
         Integer(10, 30), #name='desc_filters3'),
         Integer(10, 30), #name='desc_filters2'),
         Integer(256, 5000), #name='batch_size'),
        ]

res_gp = gbrt_minimize(train_tf, space, n_calls=100, random_state=0)

In [91]:
normll = QuantileTransformer(output_distribution='normal')
ynorm = normll.fit_transform(y)
seed=1
batchsize=1000
epochs=4
np.random.seed(seed)
tf.set_random_seed(seed)

callbacks= [ModelCheckpoint("embed_NN_"+str(seed)+".check", save_best_only=True, verbose=1)]
model = keras_mercari_model()

train_idx, val_idx= cvlist[1]
X = [X_name, X_desc, X_desc2, X_itemname, X_cats["brand_name"], X_cats["category_name"],
    X_cats["cat1"], X_cats["cat2"], X_cats["cat3"], X_cats["brand_cat"],
    X_cats["category_shipping"], X_cats["item_condition_id"], X_conts]

X_tr = [x[train_idx] for x in X]
X_val = [x[val_idx] for x in X]
model.fit(X_tr, ynorm[train_idx], batch_size=batchsize, epochs=epochs,
                               verbose=1,
                              validation_data=(X_val, ynorm[val_idx]), shuffle=True,
         )
y_val2 = y[val_idx]


(?, 7, 50)
(?, ?)
Train on 1407578 samples, validate on 74083 samples
Epoch 1/4
1407578/1407578 [==============================] - 27s 19us/step - loss: 0.5512 - val_loss: 0.4601
Epoch 2/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.4263 - val_loss: 0.4424
Epoch 3/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.4000 - val_loss: 0.4389
Epoch 4/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.3844 - val_loss: 0.4378


In [92]:
y_preds2 = normll.inverse_transform(model.predict(X_val))
print(np.sqrt(metrics.mean_squared_error(y_val2, y_preds2)))

0.42093463775873474


In [96]:
ymean = y_preds1 *0.7 + y_preds2[:, 0]*0.3 
print(np.sqrt(metrics.mean_squared_error(y_val2, ymean)))

0.40684002989865675


In [97]:
cat_means = train_data.groupby("category_name")["price"].mean()
train_data['cat_price'] = np.log1p(train_data.category_name.map(cat_means).fillna(17))
#yrel = ((train_data['price'] - train_data['cat_price'])/train_data['cat_price']).values.reshape(-1,1)
yrel = ((y[:, 0] - train_data['cat_price'])/train_data['cat_price']).values.reshape(-1,1)
normll = QuantileTransformer(output_distribution='normal')
ynorm2 = normll.fit_transform(yrel)
seed=1
batchsize=1000
epochs=4
np.random.seed(seed)
tf.set_random_seed(seed)

callbacks= [ModelCheckpoint("embed_NN_"+str(seed)+".check", save_best_only=True, verbose=1)]
model = keras_mercari_model()

train_idx, val_idx= cvlist[1]
X = [X_name, X_desc, X_desc2, X_itemname, X_cats["brand_name"], X_cats["category_name"],
    X_cats["cat1"], X_cats["cat2"], X_cats["cat3"], X_cats["brand_cat"],
    X_cats["category_shipping"], X_cats["item_condition_id"], X_conts]

X_tr = [x[train_idx] for x in X]
X_val = [x[val_idx] for x in X]
model.fit(X_tr, ynorm2[train_idx], batch_size=batchsize, epochs=epochs,
                               verbose=1,
                              validation_data=(X_val, ynorm2[val_idx]), shuffle=True,
                              )
y_val3 = y[val_idx]


(?, 7, 50)
(?, ?)
Train on 1407578 samples, validate on 74083 samples
Epoch 1/4
1407578/1407578 [==============================] - 27s 19us/step - loss: 0.5149 - val_loss: 0.4240
Epoch 2/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.3908 - val_loss: 0.4098
Epoch 3/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.3647 - val_loss: 0.4033
Epoch 4/4
1407578/1407578 [==============================] - 25s 18us/step - loss: 0.3495 - val_loss: 0.4019


In [98]:
#y_preds3 = np.log1p((normll.inverse_transform(model.predict(X_val))[:,0] + 1)* train_data['cat_price'].values[val_idx])
y_preds3 = (normll.inverse_transform(model.predict(X_val))[:,0] + 1)* train_data['cat_price'].values[val_idx]

print(np.sqrt(metrics.mean_squared_error(y_val3, y_preds3)))

0.4123020256231108


In [103]:
ymean = y_preds1 *0.4 + y_preds2[:, 0]*.2 + y_preds3 *0.4
print(np.sqrt(metrics.mean_squared_error(y_val2, ymean)))

0.40476890048691955


In [105]:
np.corrcoef(y_preds1, y_preds2[:, 0])

array([[1.        , 0.96827273],
       [0.96827273, 1.        ]])

In [ ]:
#obs:
# desc2 and itemname contribute 0.001 - 0.002
# drop rate on desc helps but not on desc2 and itemname
#try out regularization in desc
#PRelu giving 0.001 improvement
#clipping improving score a bit